# Small svd optimization - debugging

Created 19/02/2025

Objectives:
* Debug svd optimisation in previously found counterexamples.

# Package imports

In [1]:
import sys
sys.path.append('../')

In [2]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [3]:
from functools import reduce
from operator import mul

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [68]:
from SPTOptimization.SymmetryActionWithBoundaryUnitaries import SymmetryActionWithBoundaryUnitaries

from SPTOptimization.Optimizers.OneSiteSolver import OneSiteSolver

from SPTOptimization.utils import (
    to_npc_array,
    get_right_identity_environment,
    get_right_identity_environment_from_tp_tensor,
    get_left_identity_environment_from_tp_tensor,
    multiply_transfer_matrices
)

from SPTOptimization.Optimizers.utils import (
    one_site_optimization_sweep_right
)

from SPTOptimization.gradients import (
    expectation_gradient_from_environments_and_b_tensor
)

In [6]:
import re

# Load data

In [7]:
DATA_DIR = r"../data/simple_fermionic_cluster_200_site_dmrg/"

In [8]:
def parse_file_name(file_name):
    non_trivial_proj_rep = bool(int(file_name[3]))

    n1_01 = int(file_name[8])
    n1_10 = int(file_name[9])

    return (non_trivial_proj_rep, (n1_01, n1_10))

In [9]:
loaded_data = dict()

for local_file_name in os.listdir(DATA_DIR):
    f_name = r"{}/{}".format(DATA_DIR, local_file_name, ignore_unknown=False)
    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data[data_info]=data['wavefunction']

In [10]:
loaded_data

{(False, (1, 1)): <tenpy.networks.mps.MPS at 0x14ed92190>,
 (True, (1, 1)): <tenpy.networks.mps.MPS at 0x17637eb10>,
 (True, (0, 1)): <tenpy.networks.mps.MPS at 0x177614910>,
 (False, (0, 1)): <tenpy.networks.mps.MPS at 0x17764f8d0>,
 (False, (1, 0)): <tenpy.networks.mps.MPS at 0x1776d3b10>,
 (True, (1, 0)): <tenpy.networks.mps.MPS at 0x301158810>,
 (True, (0, 0)): <tenpy.networks.mps.MPS at 0x3011978d0>,
 (False, (0, 0)): <tenpy.networks.mps.MPS at 0x3011f7390>}

In [11]:
loaded_data[(True, (0,0))].expectation_value('N', sites=list(range(1, 201, 2)))

array([0.00000000e+00, 4.10310779e-55, 3.20108804e-32, 4.41166527e-30,
       4.30652209e-31, 1.28028128e-32, 2.18979328e-32, 9.64766348e-32,
       6.39204896e-31, 4.19476267e-32, 7.51835939e-33, 1.56998498e-32,
       3.06132263e-29, 6.24832887e-32, 6.64529320e-32, 6.61192056e-33,
       3.02331734e-32, 1.97077344e-32, 3.97848871e-32, 1.65936555e-32,
       8.12962747e-32, 8.36180681e-32, 1.05455111e-32, 2.30695596e-32,
       4.15719467e-32, 3.32855567e-32, 1.02762007e-32, 1.46697913e-32,
       3.80115562e-31, 1.63756131e-32, 9.98608231e-33, 1.82981182e-32,
       1.73380216e-33, 6.00451123e-32, 1.33036658e-32, 2.79091135e-31,
       1.64291033e-32, 3.68006128e-33, 9.95622775e-32, 6.17421509e-32,
       2.70280235e-32, 3.01371615e-31, 2.35183074e-32, 1.17896355e-32,
       3.93686429e-31, 8.45721273e-32, 1.08863312e-32, 3.77353007e-31,
       9.67562785e-33, 1.99656454e-33, 6.40393616e-33, 2.78084102e-32,
       1.03150150e-31, 4.37579428e-33, 1.45445289e-32, 3.55938597e-32,
      

In [12]:
loaded_data[(False, (0,0))].expectation_value('N', sites=list(range(1, 201, 2)))

array([0.00000000e+00, 3.26915052e-29, 3.33293732e-29, 0.00000000e+00,
       3.33293732e-29, 7.79030959e-29, 4.21824880e-29, 4.21824880e-29,
       5.96576060e-30, 1.14662165e-29, 3.59424750e-29, 8.18720523e-29,
       4.21824880e-29, 8.28796989e-29, 4.21824880e-29, 3.79670126e-29,
       4.21824880e-29, 8.18720523e-29, 1.37190923e-28, 0.00000000e+00,
       1.37190923e-28, 0.00000000e+00, 3.66111579e-29, 3.66111579e-29,
       3.86541844e-29, 3.33293732e-29, 0.00000000e+00, 0.00000000e+00,
       3.59424750e-29, 1.56000325e-28, 3.46235982e-29, 0.00000000e+00,
       3.59424750e-29, 3.59424750e-29, 3.72860037e-29, 0.00000000e+00,
       0.00000000e+00, 9.22258517e-29, 8.33234331e-30, 9.01057880e-29,
       3.46235982e-29, 4.43734259e-29, 3.14342582e-29, 3.14342582e-29,
       0.00000000e+00, 8.18720523e-29, 1.46709639e-29, 1.46709639e-29,
       1.46709639e-29, 8.98561875e-30, 0.00000000e+00, 8.01495006e-30,
       4.07526776e-29, 4.00470169e-29, 9.32150093e-30, 7.69262642e-29,
      

# Definitons

In [13]:
MAX_VIRTUAL_BOND_DIM = 8
MAX_INTERMEDIATE_VIRTUAL_BOND_DIM = 2*MAX_VIRTUAL_BOND_DIM
# MPO bond dim?
MAX_MPO_BOND_DIM = 50

SVD_CUTOFF = 1e-3

Define bosonic symmetries. Label by the group element added.

In [14]:
np_00 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

np_01 = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

np_10 = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

np_11 = np.array([
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0]
])

In [15]:
bosonic_np_symmetries = [
    np_00,
    np_01,
    np_10,
    np_11
]

In [16]:
npc_00 = to_npc_array(np_00)
npc_01 = to_npc_array(np_01)
npc_10 = to_npc_array(np_10)
npc_11 = to_npc_array(np_11)

Define "fermionic symmetries". Just identity and JW string.

In [17]:
np_I = np.array([
    [1, 0],
    [0, 1]
])

np_JW = np.array([
    [1, 0],
    [0, -1]
])

In [18]:
fermionic_np_symmetries = [np_I, np_JW]

In [19]:
fermionic_npc_symmetries = [
    to_npc_array(X) for X in fermionic_np_symmetries
]

In [20]:
symmetry_actions = [
    [[b, f] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [21]:
shifted_symmetry_actions = [
    [[f, b] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [22]:
left_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=1
)

In [23]:
right_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=-1
)

# Test

## Quick rdm check

In [26]:
test_psi = loaded_data[(True, (0, 0))]
test_symmetries = (npc_01, fermionic_npc_symmetries[0])

In [27]:
test_psi.L

200

In [33]:
left_symmetry_index = 50
len_symmetry_block = 100
right_symmetry_index = (
    left_symmetry_index
    + len_symmetry_block
)

In [31]:
test_psi.sites[left_symmetry_index]

ClockSite(q=4, conserve=None)

In [35]:
b = test_psi.get_B(left_symmetry_index, form='Th')
s =test_symmetries[0]
t = npc.tensordot(b, s, [['p',], ['p*',]])
t = npc.tensordot(t, b.conj(), [['p',], ['p*',]])

for i in range(left_symmetry_index + 1, right_symmetry_index + 1):
    b = test_psi.get_B(i)
    s = test_symmetries[i%2]

    t = npc.tensordot(t, b, [['vR',], ['vL',]])
    t = npc.tensordot(t, s, [['p',], ['p*',]])
    t = npc.tensordot(t, b.conj(), [['vR*', 'p'], ['vL*', 'p*']])

transfer_matrix = t

In [36]:
transfer_matrix

<npc.Array shape=(4, 4, 4, 4) labels=['vL', 'vL*', 'vR', 'vR*']>

In [38]:
grouped_transfer_matrix = (
    transfer_matrix
    .combine_legs([['vR', 'vR*'], ['vL', 'vL*']])
)

In [40]:
U, S, V = npc.svd(grouped_transfer_matrix)

In [41]:
S

array([2.50000000e-01, 1.98794041e-17, 1.65424047e-17, 1.21590815e-17,
       8.27957297e-18, 7.40972187e-18, 6.61087616e-18, 4.73116059e-18,
       4.20559138e-18, 3.44242194e-18, 2.19871759e-18, 1.32796468e-18,
       1.00075067e-18, 9.19384868e-19, 3.65421991e-19, 1.09514106e-19])

In [46]:
left_side_symmetry_env = U[:, 0].split_legs()
right_side_symmetry_env = V[0].split_legs()

In [47]:
left_side_symmetry_env

<npc.Array shape=(4, 4) labels=['vL', 'vL*']>

In [48]:
right_side_symmetry_env

<npc.Array shape=(4, 4) labels=['vR', 'vR*']>

Construct right gradient.

In [54]:
t = right_side_symmetry_env

for i in range(4):
    site_index = right_symmetry_index + 1 + i
    b = (
        test_psi
        .get_B(site_index)
        .replace_label('p', f'p{i}')
    )

    t = npc.tensordot(t, b, [['vR',], ['vL']])
    t = npc.tensordot(t, b.conj(), [['vR*',], ['vL*']])

right_env = get_right_identity_environment_from_tp_tensor(b)

right_rdm = npc.tensordot(t, right_env, [['vR', 'vR*'], ['vL', 'vL*']])

In [56]:
right_rdm

<npc.Array shape=(2, 2, 4, 4, 2, 2, 4, 4) labels=['p0', 'p0*', 'p1', 'p1*', 'p2', 'p2*', 'p3', 'p3*']>

In [58]:
grouped_right_rdm = right_rdm.combine_legs([
    ['p0', 'p1', 'p2', 'p3'],
    ['p0*', 'p1*', 'p2*', 'p3*']
])

_, S, _ = npc.svd(grouped_right_rdm)

In [60]:
np.sum(S)

1.9999999999999993

What we expect!

## Step through optimisation function, looking out for environments

In [62]:
right_bs = [
    test_psi.get_B(i) for i in range(right_symmetry_index+1, right_symmetry_index+5)
]

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [64]:
len(right_bs)

4

In [62]:
left_bs = [
    test_psi.get_B(i, form='A') for i in range(right_symmetry_index+1, right_symmetry_index+5)
]

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [65]:
left_bs = [
    test_psi.get_B(i, form='A') for i in range(left_symmetry_index-4, left_symmetry_index)
]

In [66]:
len(left_bs)

4

In [67]:
transfer_matrix

<npc.Array shape=(4, 4, 4, 4) labels=['vL', 'vL*', 'vR', 'vR*']>

In [76]:
left_identity_environment = get_left_identity_environment_from_tp_tensor(left_bs[0])

t = left_identity_environment

for i, b in enumerate(left_bs):
    w = b.copy().replace_label('p', f'p{i}')
    t = npc.tensordot(t, w, [['vR',], ['vL',]])
    t = npc.tensordot(t, w.conj(), [['vR*',], ['vL*',]])

t = t.combine_legs([
    ['p0', 'p1', 'p2', 'p3'],
    ['p0*', 'p1*', 'p2*', 'p3*']
])

t = t.replace_labels(
    ['(p0.p1.p2.p3)', '(p0*.p1*.p2*.p3*)'],
    ['p', 'p*']
)

left_partial_rdm = t

In [77]:
left_partial_rdm

<npc.Array shape=(64, 64, 4, 4) labels=['p', 'p*', 'vR', 'vR*']>

In [79]:
right_identity_environment = get_right_identity_environment_from_tp_tensor(right_bs[-1])

t = right_identity_environment

for i, b in enumerate(right_bs[::-1]):
    w = b.copy().replace_label('p', f'p{i}')
    t = npc.tensordot(t, w, [['vL',], ['vR',]])
    t = npc.tensordot(t, w.conj(), [['vL*',], ['vR*',]])

t = t.combine_legs([
    ['p0', 'p1', 'p2', 'p3'],
    ['p0*', 'p1*', 'p2*', 'p3*']
])

t = t.replace_labels(
    ['(p0.p1.p2.p3)', '(p0*.p1*.p2*.p3*)'],
    ['p', 'p*']
)

right_partial_rdm = t

In [80]:
right_partial_rdm

<npc.Array shape=(64, 64, 4, 4) labels=['p', 'p*', 'vL', 'vL*']>

In [81]:
np_identity = np.identity(64, dtype=np.complex128)

In [82]:
left_u = to_npc_array(np_identity)
right_u = to_npc_array(np_identity)

In [83]:
t = left_partial_rdm

t = npc.tensordot(t, left_u, [['p', 'p*'], ['p*', 'p']])
t = npc.tensordot(t, transfer_matrix, [['vR', 'vR*'], ['vL', 'vL*']])
t = npc.tensordot(t, right_partial_rdm, [['vR', 'vR*'], ['vL', 'vL*']])

right_rdm = t

In [84]:
t

<npc.Array shape=(64, 64) labels=['p', 'p*']>

In [92]:
_, S, _ = npc.svd(right_rdm)

In [93]:
np.sum(S)

3.2738944750195058e-15

In [89]:
t = right_partial_rdm

t = npc.tensordot(t, right_u, [['p', 'p*'], ['p*', 'p']])
t = npc.tensordot(t, transfer_matrix, [['vL', 'vL*'], ['vR', 'vR*']])
t = npc.tensordot(t, left_partial_rdm, [['vL', 'vL*'], ['vR', 'vR*']])

left_rdm = t

In [90]:
_, S, _ = npc.svd(left_rdm)

In [91]:
np.sum(S)

1.2655020850241507e-14

In [110]:
U, S, V = npc.svd(left_rdm, inner_labels = ['p*', 'p'])

In [111]:
U

<npc.Array shape=(64, 64) labels=['p', 'p*']>

In [112]:
V

<npc.Array shape=(64, 64) labels=['p', 'p*']>

In [113]:
new_left_u = npc.tensordot(U, V, [['p*',], ['p',]])
new_left_u = new_left_u.conj()

In [114]:
new_left_u

<npc.Array shape=(64, 64) labels=['p*', 'p']>

In [115]:
npc.tensordot(left_rdm, new_left_u, [['p', 'p*'], ['p*', 'p']])

(1.2655020850241507e-14+0j)

Close enough. Now back to the left side.

In [116]:
t = left_partial_rdm

t = npc.tensordot(t, new_left_u, [['p', 'p*'], ['p*', 'p']])
t = npc.tensordot(t, transfer_matrix, [['vR', 'vR*'], ['vL', 'vL*']])
t = npc.tensordot(t, right_partial_rdm, [['vR', 'vR*'], ['vL', 'vL*']])

right_rdm = t

In [117]:
_, S, _ = npc.svd(right_rdm)

In [118]:
np.sum(S)

0.9999976469864238

In [119]:
U, S, V = npc.svd(right_rdm, inner_labels = ['p*', 'p'])

In [120]:
U

<npc.Array shape=(64, 64) labels=['p', 'p*']>

In [121]:
V

<npc.Array shape=(64, 64) labels=['p', 'p*']>

In [122]:
new_right_u = npc.tensordot(U, V, [['p*',], ['p',]])
new_right_u = new_right_u.conj()

In [123]:
new_right_u

<npc.Array shape=(64, 64) labels=['p*', 'p']>

## Functions

### Leg and label functions

In [124]:
P_LEG_LABEL_REGEX_STRING = r"^p\d*$"
p_leg_pattern = re.compile(P_LEG_LABEL_REGEX_STRING)

def is_single_physical_leg_label(label):
    return bool(p_leg_pattern.match(label))

In [125]:
def is_grouped_physical_leg_label(leg_label):
    head, *body, tail = leg_label
    if (head != '(') or (tail != ')'):
        return False

    body = ''.join(body)
    legs = body.split('.')

    return all(is_single_physical_leg_label(l) for l in legs)

In [126]:
def is_physical_leg_label(label):
    out = (
        is_single_physical_leg_label(label)
        or is_grouped_physical_leg_label(label)
    )

    return out

In [127]:
def extract_single_physical_leg_label_from_tensor(b):
    out = next(
        l for l in b.get_leg_labels()
        if is_single_physical_leg_label(l)
    )

    return out

In [128]:
def conjugate_single_physical_leg_label(label):
    return label + '*'

In [129]:
def conjugate_grouped_physical_leg_label(label):
    _, *body, _ = label
    single_labels = ''.join(body).split('.')

    conjugate_labels = [
        conjugate_single_physical_leg_label(l)
        for l in single_labels
    ]

    conjugate_body = '.'.join(conjugate_labels)
    out = '(' + conjugate_body + ')'
    
    return out

In [130]:
def conjugate_leg_label(label):
    if is_single_physical_leg_label(label):
        return conjugate_single_physical_leg_label(label)
    elif is_grouped_physical_leg_label(label):
        return conjugate_grouped_physical_leg_label(label)
    else:
        raise ValueError

In [131]:
def get_physical_leg_labels(t):
    out = [
        ll for ll in t.get_leg_labels()
        if is_physical_leg_label(ll)
    ]

    return out

### Other

In [132]:
def contract_virtual_legs(tl, tr):
    return npc.tensordot(tl, tr, ['vR', 'vL'])

In [133]:
def combine_tensors(tensors):
    out = reduce(contract_virtual_legs, tensors)

    leg_labels = [
        extract_single_physical_leg_label_from_tensor(t)
        for t in tensors
    ]

    out = out.combine_legs(leg_labels)

    return out

In [134]:
def combine_b_tensors(b_tensors):
    renamed_tensors = [
        b.replace_label('p', f'p{i}')
        for i, b in enumerate(b_tensors)
    ]

    return combine_tensors(renamed_tensors)

In [135]:
def get_identity_operator(mps_tensor):
    p_leg_label = get_physical_leg_labels(mps_tensor)[0]
    p_leg = mps_tensor.get_leg(p_leg_label)
    p_leg_label_conj = conjugate_leg_label(p_leg_label)

    out = npc.diag(
        1,
        leg=p_leg,
        dtype='complex',
        labels=[p_leg_label, p_leg_label_conj]
    )

    return out

In [136]:
def svd_reduce_split_tensor(t, max_inner_dim=MAX_VIRTUAL_BOND_DIM,
                           normalise=True, svd_cutoff=SVD_CUTOFF):
    U, S, VH = npc.svd(
        t,
        compute_uv=True,
        inner_labels=['vR', 'vL'],
        cutoff=svd_cutoff
    )

    # Truncate tensors:
    U = U[:, :max_inner_dim]
    S = S[:max_inner_dim]
    VH = VH[:max_inner_dim, :]

    if normalise:
        new_norm = np.sqrt(np.sum(S**2))
        S = S/new_norm

    """
    leg = VH.get_leg('vL')

    schmidt_values = npc.diag(S, leg, labels=['vL', 'vR'])
    """

    return U, S, VH

In [137]:
def split_combined_b(b, leftmost_schmidt_values,
                     max_virtual_bond_dim=MAX_INTERMEDIATE_VIRTUAL_BOND_DIM,
                     p_leg_labels=None):
    t = b.split_legs()

    num_sites = t.ndim - 2

    if p_leg_labels is None:
        p_leg_labels = [f'p{i}' for i in range(num_sites)]

    out_bs = list()
    out_schmidt_values = list()

    current_left_schmidt_values = leftmost_schmidt_values

    for i, ll in enumerate(p_leg_labels[:-1]):
        # In case the bond dimension has been truncated. May need to add in a
        # case if have less schmidt values than the bond dim...
        bond_dim = t.get_leg('vL').get_block_sizes()[0]
        t.iscale_axis(current_left_schmidt_values[:bond_dim], axis='vL')

        tail_legs = p_leg_labels[(i+1):]
        
        t = t.combine_legs([['vL', ll], ['vR', *tail_legs]])

        U, S, VH = svd_reduce_split_tensor(
            t,
            max_inner_dim=max_virtual_bond_dim,
            normalise=True
        )

        bl = (
            U
            .split_legs()
            .replace_label(ll, 'p')
        )
        bl.iscale_axis(1/current_left_schmidt_values[:bond_dim], axis='vL')
        bl.iscale_axis(S, axis='vR')
        bl.itranspose(['vL', 'p', 'vR'])
        out_bs.append(bl)

        out_schmidt_values.append(S)
        current_left_schmidt_values=S

        t = VH.split_legs()

    bl = t.replace_label(p_leg_labels[-1], 'p')
    bl.itranspose(['vL', 'p', 'vR'])
    out_bs.append(bl)

    return out_bs, out_schmidt_values

In [138]:
def split_b(b, max_virtual_bond_dim=MAX_INTERMEDIATE_VIRTUAL_BOND_DIM,
                     p_leg_labels=None):
    leg_label = get_physical_leg_labels(b)[0]

    if is_single_physical_leg_label(leg_label):
        return b
    elif is_grouped_physical_leg_label(leg_label):
        return split_combined_b(b, max_virtual_bond_dim, p_leg_labels)
    else:
        raise ValueError

In [139]:
def split_combined_u(u, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM,
                     p_leg_labels=None):
    t = u.split_legs()
    num_sites = t.ndim //2
    
    if p_leg_labels is None:
        p_leg_labels = [f'p{i}' for i in range(num_sites)]

    p_leg_labels_hc = [l+'*' for l in p_leg_labels]
    out_ws = list()

    for i, (ll, ll_hc) in enumerate(zip(p_leg_labels[:-1], p_leg_labels_hc[:-1])):
        tail_legs = p_leg_labels[(i+1):]
        tail_legs_hc = p_leg_labels_hc[(i+1):]

        left_leg_labels = [ll, ll_hc]
        if 'vL' in t.get_leg_labels():
            left_leg_labels.append('vL')
    
        t = t.combine_legs([left_leg_labels, [*tail_legs, *tail_legs_hc]])

        U, S, VH = svd_reduce_split_tensor(
            t,
            max_inner_dim=max_virtual_bond_dim,
            normalise=False
        )

        U.iscale_axis(S, axis='vR')

        wl = U.split_legs().replace_labels([ll, ll_hc], ['p', 'p*'])
        #wl.itranspose(['p', 'p*'])
        out_ws.append(wl)

        t = VH.split_legs()

    wl = t.replace_labels(
        [p_leg_labels[-1], p_leg_labels_hc[-1]],
        ['p', 'p*']
    )
    #wl.itranspose(['p', 'p*'])
    out_ws.append(wl)

    out_ws[0] = (
        out_ws[0]
        .add_leg(left_trivial_leg_charge, 0, label='vL')
    )
    
    out_ws[-1] = (
        out_ws[-1]
        .add_leg(right_trivial_leg_charge, 0, label='vR')
    )

    return out_ws

In [140]:
def split_u(u, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM,
            p_leg_labels=None):
    leg_label = get_physical_leg_labels(u)[0]

    if is_single_physical_leg_label(leg_label):
        return [unitary_to_mpo_tensor(u),]
    elif is_grouped_physical_leg_label(leg_label):
        return split_combined_u(u, max_virtual_bond_dim, p_leg_labels)
    else:
        raise ValueError

In [141]:
def unitary_to_mpo_tensor(u):
    out = (
        u
        .add_leg(left_trivial_leg_charge, 0, label='vL')
        .add_leg(right_trivial_leg_charge, 0, label='vR')
    )

    return out

In [142]:
def multiply_individual_mpo_tensors(w1, w2):
    w1 = w1.replace_labels(['vR', 'vL'], ['vR1', 'vL1'])
    w2 = w2.replace_labels(['vR', 'vL'], ['vR2', 'vL2'])

    w = (
        npc.tensordot(w1, w2, [['p'], ['p*']])
        .combine_legs([['vR1', 'vR2'], ['vL1', 'vL2']])
        .replace_labels(['(vR1.vR2)', '(vL1.vL2)'], ['vR', 'vL'])
    )

    return w

In [143]:
def group_elements(l, group_size, offset=0):
    first, rest = l[:offset], l[offset:]

    num_rest_groups = ((len(rest)-1)//group_size) + 1

    groups = [first,] if first else list()

    for i in range(num_rest_groups):
        first_index = i*group_size
        last_index = (i+1)*group_size
        groups.append(rest[first_index:last_index])

    return groups

In [144]:
def combine_grouped_b_tensors(grouped_bs):
    out = list()

    for group in grouped_bs:
        if len(group) == 1:
            out.append(group[0])
        else:
            out.append(combine_b_tensors(group))

    return out

In [145]:
def reduce_virtual_mpo_dim(wl, wr, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM):
    # wl = wl.replace_labels(['p', 'p*'], ['pl', 'pl*'])

    wl = wl.combine_legs(['p', 'vL', 'p*'])
    wr = wr.combine_legs(['p', 'vR', 'p*'])
    w = npc.tensordot(wl, wr, ['vR', 'vL'])

    wl, S, wr = svd_reduce_split_tensor(
        w,
        max_inner_dim=max_virtual_bond_dim
    )

    wl.iscale_axis(S, axis='vR')

    wl = wl.split_legs()
    wr = wr.split_legs()

    return wl, wr

In [146]:
def multiply_mpos(top_ws, bottom_ws, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM):
    ws = list()
    
    for w1, w2 in zip(top_ws, bottom_ws):
        w = multiply_individual_mpo_tensors(w1, w2)
        ws.append(w)


    for i in range(len(ws)-1):
        ws[i], ws[i+1] = reduce_virtual_mpo_dim(
            ws[i],
            ws[i+1],
            max_virtual_bond_dim=max_virtual_bond_dim
        )

    return ws

### Layer function

In [147]:
def optimise_layer(
        left_environment,
        top_b_tensors,
        left_schmidt_values,
        block_width,
        block_offset,
        expectations,
        unitaries=list(),
        bottom_b_tensors=None,
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_b_tensors is None:
        bottom_b_tensors = top_b_tensors

    group = lambda x: group_elements(x, block_width, block_offset)
    top_grouped_bs = group(top_b_tensors)
    bottom_grouped_bs = group(bottom_b_tensors)
    grouped_schmidt_values = group(left_schmidt_values)

    top_combined_bs = combine_grouped_b_tensors(top_grouped_bs)
    bottom_combined_bs = combine_grouped_b_tensors(bottom_grouped_bs)

    new_unitaries = [
        get_identity_operator(t) for t in top_combined_bs
    ]

    expectations.append(list())

    for _ in range(num_iterations):
        exps, *_ = one_site_optimization_sweep_right(
            left_environment,
            top_combined_bs,
            new_unitaries,
            bottom_combined_bs
        )

        expectations[-1].append(exps)

    unitaries.append(new_unitaries)

    for i, u in enumerate(new_unitaries):
        b = top_combined_bs[i]
        ll = get_physical_leg_labels(b)[0]
        llh = conjugate_leg_label(ll)
    
        new_b = npc.tensordot(b, u, [[ll,], [llh,]])
    
        top_combined_bs[i] = new_b

    new_top_bs = list()
    #new_left_schmidt_values = left_schmidt_values.copy()
    new_left_schmidt_values = list()

    for b, s in zip (top_combined_bs, grouped_schmidt_values):
        """
        print(b)
        print(s)
        print('\n')
        """
        leg_label = get_physical_leg_labels(b)[0]
        if is_single_physical_leg_label(leg_label):
            new_top_bs.append(b)
            new_left_schmidt_values.extend(s)
        elif is_grouped_physical_leg_label(leg_label):
            bs, schmidt_vals = split_combined_b(
                b,
                s[0],
                max_virtual_bond_dim
            )
            new_top_bs.extend(bs)
            new_left_schmidt_values.extend(s)

    """
    for i, b in enumerate(new_top_bs):
        top_b_tensors[i] = b
    """

    return new_top_bs, new_left_schmidt_values

In [148]:
def inner_product_b_tensors(b_tensors, b_bra_tensors=None, left_environment=None,
                            right_environment=None):
    if b_bra_tensors is None:
        b_bra_tensors = b_tensors

    b = b_tensors[0]
    b_bra = b_bra_tensors[0]

    if left_environment is None:
        t = npc.tensordot(b, b_bra.conj(), [['vL',], ['vL*',]])
    else:
        t = npc.tensordot(left_environment, b, [['vR',], ['vL',]])
        t = npc.tensordot(t, b_bra.conj(), [['vR*', 'p'], ['vL*', 'p*']])

    for b, b_bra in zip(b_tensors[1:], b_bra_tensors[1:]):
        t = npc.tensordot(t, b, [['vR',], ['vL',]])
        t = npc.tensordot(t, b_bra.conj(), [['vR*', 'p'], ['vL*', 'p*']])

    if right_environment is None:
        out = npc.trace(t)
    else:
        out = npc.tensordot(t, right_environment, [['vR', 'vR*'], ['vL', 'vL*']])

    return out

In [149]:
def get_left_side_right_symmetry_environment(
    right_top_b_tensors, right_bottom_b_tensors, symmetry_transfer_matrix
    ):

    if right_bottom_b_tensors is None:
        right_bottom_b_tensors = right_top_b_tensors

    t = get_right_identity_environment_from_tp_tensor(right_top_b_tensors[-1])

    for tb, bb in zip(right_top_b_tensors[::-1], right_bottom_b_tensors[::-1]):
        t = npc.tensordot(t, tb, [['vL',], ['vR']])
        t = npc.tensordot(t, bb.conj(), [['vL*', 'p'], ['vR*', 'p*']])

    t = npc.tensordot(
        t,
        symmetry_transfer_matrix,
        [['vL', 'vL*',], ['vR', 'vR*']]
    )

    return t

In [150]:
def swap_left_right_indices(npc_array):
    left_right_pairs = {
        'vL': 'vR',
        'vR': 'vL',
        'vL*': 'vR*',
        'vR*': 'vL*'
    }

    leg_labels = npc_array.get_leg_labels()

    old_labels = [l for l in leg_labels if l in left_right_pairs]
    new_labels = [left_right_pairs[l] for l in old_labels]

    out = npc_array.replace_labels(old_labels, new_labels)

    return out

In [151]:
def two_sided_optimise_layer(
        symmetry_transfer_matrix,
        top_left_b_tensors,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        left_expectations,
        bottom_left_b_tensors=None,
        bottom_right_b_tensors=None,
        left_unitaries=list(),
        right_unitaries=list(),
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_left_b_tensors is None:
        bottom_left_b_tensors = top_left_b_tensors

    if bottom_right_b_tensors is None:
        bottom_right_b_tensors = top_right_b_tensors

    right_side_left_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            top_left_b_tensors,
            bottom_left_b_tensors,
            swap_left_right_indices(symmetry_transfer_matrix)
        )
    )

    right_side_left_symmetry_environment = swap_left_right_indices(
        right_side_left_symmetry_environment
    )
    
    new_right_pair = optimise_layer(
        right_side_left_symmetry_environment,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        right_unitaries,
        bottom_right_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_right_b_tensors = new_right_pair[0]
    new_right_side_left_schmidt_values = new_right_pair[1]

    left_side_right_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            new_top_right_b_tensors,
            bottom_right_b_tensors,
            symmetry_transfer_matrix
        )
    )

    left_side_right_symmetry_environment = swap_left_right_indices(
        left_side_right_symmetry_environment
    )

    new_left_pair = optimise_layer(
        left_side_right_symmetry_environment,
        top_left_b_tensors,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        left_expectations,
        left_unitaries,
        bottom_left_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_left_b_tensors = new_left_pair[0]
    new_left_side_right_schmidt_values = new_left_pair[1]

    return (new_left_pair, new_right_pair)

In [152]:
def initialize_brick_optimisation(symmetry_case, num_sites):
    symmetry_transfer_matrix = symmetry_case.npc_symmetry_transfer_matrix

    right_site_indices = list(range(
        symmetry_case.right_symmetry_index + 1,
        symmetry_case.right_symmetry_index + 1 + num_sites
    ))

    right_mps_tensors = [
        symmetry_case.psi.get_B(i)
        for k, i in enumerate(right_site_indices)
    ]

    right_side_left_schmidt_values = [
        symmetry_case.psi.get_SL(i)
        for i in right_site_indices
    ]

    left_site_indices = list(range(
        symmetry_case.left_symmetry_index - 1,
        symmetry_case.left_symmetry_index - 1 - num_sites,
        -1
    ))

    left_mps_tensors = [
        symmetry_case.psi.get_B(i, form='A')
        for k, i in enumerate(left_site_indices)
    ]
    
    left_mps_tensors = [swap_left_right_indices(b) for b in left_mps_tensors]

    left_side_right_schmidt_values = [
        symmetry_case.psi.get_SR(i)
        for i in left_site_indices
    ]

    right_expectations=list()
    left_expectations=list()
    left_unitaries=list()
    right_unitaries=list()

    return (
        symmetry_transfer_matrix,
        left_mps_tensors,
        right_mps_tensors,
        right_side_left_schmidt_values,
        left_side_right_schmidt_values,
        right_expectations,
        left_expectations,
        left_unitaries,
        right_unitaries
    )

### Output gradient tensor "one site" functions

In [153]:
def optimise_layer_with_gradient(
        left_environment,
        top_b_tensors,
        left_schmidt_values,
        block_width,
        block_offset,
        expectations,
        unitaries=list(),
        bottom_b_tensors=None,
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_b_tensors is None:
        bottom_b_tensors = top_b_tensors

    group = lambda x: group_elements(x, block_width, block_offset)
    top_grouped_bs = group(top_b_tensors)
    bottom_grouped_bs = group(bottom_b_tensors)
    grouped_schmidt_values = group(left_schmidt_values)

    top_combined_bs = combine_grouped_b_tensors(top_grouped_bs)
    bottom_combined_bs = combine_grouped_b_tensors(bottom_grouped_bs)

    new_unitaries = [
        get_identity_operator(t) for t in top_combined_bs
    ]

    expectations.append(list())

    # Below code is a hack to add in the gradient, assuming one block.
    right_environment = get_right_identity_environment_from_tp_tensor(
        top_b_tensors[-1]
    )

    grad = expectation_gradient_from_environments_and_b_tensor(
        top_combined_bs[0],
        left_environment,
        right_environment,
        bottom_combined_bs[0]
    )
    # Hack ends here.

    for _ in range(num_iterations):
        exps, *_ = one_site_optimization_sweep_right(
            left_environment,
            top_combined_bs,
            new_unitaries,
            bottom_combined_bs
        )

        expectations[-1].append(exps)

    unitaries.append(new_unitaries)

    for i, u in enumerate(new_unitaries):
        b = top_combined_bs[i]
        ll = get_physical_leg_labels(b)[0]
        llh = conjugate_leg_label(ll)
    
        new_b = npc.tensordot(b, u, [[ll,], [llh,]])
    
        top_combined_bs[i] = new_b

    new_top_bs = list()
    #new_left_schmidt_values = left_schmidt_values.copy()
    new_left_schmidt_values = list()

    for b, s in zip (top_combined_bs, grouped_schmidt_values):
        """
        print(b)
        print(s)
        print('\n')
        """
        leg_label = get_physical_leg_labels(b)[0]
        if is_single_physical_leg_label(leg_label):
            new_top_bs.append(b)
            new_left_schmidt_values.extend(s)
        elif is_grouped_physical_leg_label(leg_label):
            bs, schmidt_vals = split_combined_b(
                b,
                s[0],
                max_virtual_bond_dim
            )
            new_top_bs.extend(bs)
            new_left_schmidt_values.extend(s)

    """
    for i, b in enumerate(new_top_bs):
        top_b_tensors[i] = b
    """

    return new_top_bs, new_left_schmidt_values, grad

In [154]:
def two_sided_optimise_layer_with_gradient(
        symmetry_transfer_matrix,
        top_left_b_tensors,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        left_expectations,
        bottom_left_b_tensors=None,
        bottom_right_b_tensors=None,
        left_unitaries=list(),
        right_unitaries=list(),
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_left_b_tensors is None:
        bottom_left_b_tensors = top_left_b_tensors

    if bottom_right_b_tensors is None:
        bottom_right_b_tensors = top_right_b_tensors

    right_side_left_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            top_left_b_tensors,
            bottom_left_b_tensors,
            swap_left_right_indices(symmetry_transfer_matrix)
        )
    )

    right_side_left_symmetry_environment = swap_left_right_indices(
        right_side_left_symmetry_environment
    )
    
    new_right_triple = optimise_layer_with_gradient(
        right_side_left_symmetry_environment,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        right_unitaries,
        bottom_right_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_right_b_tensors = new_right_triple[0]
    new_right_side_left_schmidt_values = new_right_triple[1]
    new_right_grad = new_right_triple[2]

    left_side_right_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            new_top_right_b_tensors,
            bottom_right_b_tensors,
            symmetry_transfer_matrix
        )
    )

    left_side_right_symmetry_environment = swap_left_right_indices(
        left_side_right_symmetry_environment
    )

    new_left_triple = optimise_layer_with_gradient(
        left_side_right_symmetry_environment,
        top_left_b_tensors,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        left_expectations,
        left_unitaries,
        bottom_left_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_left_b_tensors = new_left_triple[0]
    new_left_side_right_schmidt_values = new_left_triple[1]
    new_left_grad = new_left_triple[2]

    return (new_left_triple, new_right_triple)

# Step through
Step through otpimization to see what's going wrong.

In [156]:
cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                s*50,
                left_symmetry_index=50,
                left_boundary_unitaries=[np_I, np_00]*1,
                right_boundary_unitaries=[np_00, np_I]*1
            )

            cases[(*k, i, j)] = case

In [161]:
for c in cases.values():
    c.compute_svd_approximate_expectation()

In [199]:
test_case = cases[True, (0, 0), 0, 1]

In [158]:
test_case

## Quick run

In [200]:
block_width = 4
num_iterations = 1
num_layers = 1
num_sites = 4

In [206]:
c = test_case

(
    symmetry_transfer_matrix,
    current_left_mps_tensors,
    current_right_mps_tensors,
    current_right_side_left_schmidt_values,
    current_left_side_right_schmidt_values,
    current_right_expectations,
    current_left_expectations,
    current_left_unitaries,
    current_right_unitaries
) = initialize_brick_optimisation(c, num_sites)

top_right_mps_tensors = [current_right_mps_tensors,]
top_left_mps_tensors = [current_left_mps_tensors,]

right_side_left_schmidt_values_list = [current_right_side_left_schmidt_values,]
left_side_right_schmidt_values_list = [current_left_side_right_schmidt_values,]

out_quad = two_sided_optimise_layer(
    symmetry_transfer_matrix,
    top_left_mps_tensors[-1],
    top_right_mps_tensors[-1],
    right_side_left_schmidt_values_list[-1],
    left_side_right_schmidt_values_list[-1],
    block_width,
    block_offset,
    current_right_expectations,
    current_left_expectations,
    current_left_mps_tensors,
    current_right_mps_tensors,
    current_left_unitaries,
    current_right_unitaries,
    num_iterations
)

left_pair, right_pair = out_quad

top_right_mps_tensors.append(right_pair[0])
right_side_left_schmidt_values_list.append(right_pair[1])

top_left_mps_tensors.append(left_pair[0])
left_side_right_schmidt_values_list.append(left_pair[1])


out_quad = two_sided_optimise_layer(
    symmetry_transfer_matrix,
    top_left_mps_tensors[-1],
    top_right_mps_tensors[-1],
    right_side_left_schmidt_values_list[-1],
    left_side_right_schmidt_values_list[-1],
    block_width,
    block_offset,
    current_right_expectations,
    current_left_expectations,
    current_left_mps_tensors,
    current_right_mps_tensors,
    current_left_unitaries,
    current_right_unitaries,
    num_iterations
)

left_pair, right_pair = out_quad

top_right_mps_tensors.append(right_pair[0])
right_side_left_schmidt_values_list.append(right_pair[1])

top_left_mps_tensors.append(left_pair[0])
left_side_right_schmidt_values_list.append(left_pair[1])

In [207]:
current_right_expectations

[[[3.211214886573137e-15]], [[0.9999999999999987]]]

In [208]:
current_left_expectations

[[[0.9999349351458662]], [[0.9999999999999983]]]

In [162]:
(
    symmetry_transfer_matrix,
    current_left_mps_tensors,
    current_right_mps_tensors,
    current_right_side_left_schmidt_values,
    current_left_side_right_schmidt_values,
    current_right_expectations,
    current_left_expectations,
    current_left_unitaries,
    current_right_unitaries
) = initialize_brick_optimisation(test_case, num_sites)

In [163]:
symmetry_transfer_matrix

<npc.Array shape=(4, 4, 4, 4) labels=['vL*', 'vL', 'vR', 'vR*']>

In [164]:
current_left_mps_tensors

[<npc.Array shape=(4, 2, 4) labels=['vR', 'p', 'vL']>,
 <npc.Array shape=(4, 4, 4) labels=['vR', 'p', 'vL']>,
 <npc.Array shape=(4, 2, 4) labels=['vR', 'p', 'vL']>,
 <npc.Array shape=(4, 4, 4) labels=['vR', 'p', 'vL']>]

In [165]:
current_right_mps_tensors

[<npc.Array shape=(4, 4, 4) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(4, 2, 4) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(4, 4, 4) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(4, 2, 4) labels=['vL', 'p', 'vR']>]

In [166]:
current_right_side_left_schmidt_values

[array([0.5, 0.5, 0.5, 0.5]),
 array([0.5, 0.5, 0.5, 0.5]),
 array([0.5, 0.5, 0.5, 0.5]),
 array([0.5, 0.5, 0.5, 0.5])]

In [168]:
current_left_side_right_schmidt_values

[array([0.5, 0.5, 0.5, 0.5]),
 array([0.5, 0.5, 0.5, 0.5]),
 array([0.5, 0.5, 0.5, 0.5]),
 array([0.5, 0.5, 0.5, 0.5])]

In [169]:
current_right_expectations

[]

In [170]:
current_left_expectations

[]

In [171]:
current_left_unitaries

[]

In [172]:
current_right_unitaries

[]

In [173]:
top_right_mps_tensors = [current_right_mps_tensors,]
top_left_mps_tensors = [current_left_mps_tensors,]

In [174]:
top_right_mps_tensors

[[<npc.Array shape=(4, 4, 4) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(4, 2, 4) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(4, 4, 4) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(4, 2, 4) labels=['vL', 'p', 'vR']>]]

In [175]:
top_left_mps_tensors

[[<npc.Array shape=(4, 2, 4) labels=['vR', 'p', 'vL']>,
  <npc.Array shape=(4, 4, 4) labels=['vR', 'p', 'vL']>,
  <npc.Array shape=(4, 2, 4) labels=['vR', 'p', 'vL']>,
  <npc.Array shape=(4, 4, 4) labels=['vR', 'p', 'vL']>]]

In [176]:
right_side_left_schmidt_values_list = [current_right_side_left_schmidt_values,]
left_side_right_schmidt_values_list = [current_left_side_right_schmidt_values,]

In [177]:
block_offset = (block_width//2)*i % block_width

In [178]:
block_offset

1

In [179]:
top_left_b_tensors=top_left_mps_tensors[-1]
top_right_b_tensors=top_right_mps_tensors[-1]
right_side_left_schmidt_values=right_side_left_schmidt_values_list[-1]
left_side_right_schmidt_values=left_side_right_schmidt_values_list[-1]
right_expectations=current_right_expectations
left_expectations=current_left_expectations
bottom_left_b_tensors=current_left_mps_tensors
bottom_right_b_tensors=current_right_mps_tensors
left_unitaries=current_left_unitaries
right_unitaries=current_right_unitaries
num_iterations=1
max_virtual_bond_dim=8

In [180]:
right_side_left_symmetry_environment = (
    get_left_side_right_symmetry_environment(
        top_left_b_tensors,
        bottom_left_b_tensors,
        swap_left_right_indices(symmetry_transfer_matrix)
    )
)

In [182]:
right_side_left_symmetry_environment.to_ndarray()

array([[ 3.21261973e-15+0.j, -4.55586983e-15+0.j,  1.31746363e-16+0.j,
        -6.90977863e-16+0.j],
       [-4.56974762e-15+0.j, -2.99808130e-15+0.j, -1.07721990e-16+0.j,
        -1.30493035e-15+0.j],
       [ 1.29361118e-16+0.j, -1.10107234e-16+0.j, -5.63122964e-15+0.j,
         2.72218568e-16+0.j],
       [-6.87508416e-16+0.j, -1.30146091e-15+0.j,  2.72218568e-16+0.j,
         5.42363011e-15+0.j]])

In [185]:
right_side_left_symmetry_environment = swap_left_right_indices(
    right_side_left_symmetry_environment
)

In [183]:
left_side_right_symmetry_environment = (
    get_left_side_right_symmetry_environment(
        top_right_b_tensors,
        bottom_right_b_tensors,
        symmetry_transfer_matrix
    )
)

In [184]:
left_side_right_symmetry_environment.to_ndarray()

array([[-5.23067003e-16+0.j, -1.81063711e-17+0.j, -1.29914888e-16+0.j,
         2.38773232e-18+0.j],
       [-1.81063711e-17+0.j,  4.97289423e-16+0.j, -1.93681667e-18+0.j,
         1.71532118e-17+0.j],
       [-1.29914888e-16+0.j, -1.90971162e-18+0.j,  5.36933521e-16+0.j,
        -1.88415266e-17+0.j],
       [ 2.38773232e-18+0.j,  1.71532118e-17+0.j, -1.88415266e-17+0.j,
        -4.90339259e-16+0.j]])

In [186]:
new_right_pair = optimise_layer(
    right_side_left_symmetry_environment,
    top_right_b_tensors,
    right_side_left_schmidt_values,
    block_width,
    block_offset,
    right_expectations,
    right_unitaries,
    bottom_right_b_tensors,
    num_iterations,
    max_virtual_bond_dim
)

new_top_right_b_tensors = new_right_pair[0]
new_right_side_left_schmidt_values = new_right_pair[1]

In [187]:
new_top_right_b_tensors

[<npc.Array shape=(4, 4, 4) labels=['vL', 'vR', 'p']>,
 <npc.Array shape=(4, 2, 4) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(4, 4, 4) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(4, 4, 2) labels=['vL', 'vR', 'p']>]

In [188]:
new_right_side_left_schmidt_values

[array([0.5, 0.5, 0.5, 0.5]),
 array([0.5, 0.5, 0.5, 0.5]),
 array([0.5, 0.5, 0.5, 0.5]),
 array([0.5, 0.5, 0.5, 0.5])]

In [189]:
left_side_right_symmetry_environment = (
    get_left_side_right_symmetry_environment(
        new_top_right_b_tensors,
        bottom_right_b_tensors,
        symmetry_transfer_matrix
    )
)

In [190]:
left_side_right_symmetry_environment.to_ndarray()

array([[ 1.72732703e-14+0.j,  6.11011431e-16+0.j,  4.56727392e-15+0.j,
        -8.08078494e-17+0.j],
       [ 6.11011431e-16+0.j, -1.77764499e-14+0.j,  6.77732861e-17+0.j,
        -5.85008877e-16+0.j],
       [ 4.57061988e-15+0.j,  6.78546013e-17+0.j, -1.72129746e-14+0.j,
         6.97275977e-16+0.j],
       [-8.08078494e-17+0.j, -5.84947136e-16+0.j,  6.97275977e-16+0.j,
         1.77927637e-14+0.j]])

In [191]:
left_side_right_symmetry_environment = swap_left_right_indices(
    left_side_right_symmetry_environment
)

In [192]:
new_left_pair = optimise_layer(
    left_side_right_symmetry_environment,
    top_left_b_tensors,
    left_side_right_schmidt_values,
    block_width,
    block_offset,
    left_expectations,
    left_unitaries,
    bottom_left_b_tensors,
    num_iterations,
    max_virtual_bond_dim
)

In [193]:
left_expectations

[[[7.131365778204419e-14, 7.131365778204419e-14, 7.13136577820442e-14]]]

In [ ]:
out_quad = two_sided_optimise_layer(
    symmetry_transfer_matrix,
    top_left_mps_tensors[-1],
    top_right_mps_tensors[-1],
    ,
    ,
    block_width,
    block_offset,
    current_right_expectations,
    current_left_expectations,
    current_left_mps_tensors,
    current_right_mps_tensors,
    current_left_unitaries,
    current_right_unitaries,
    num_iterations
)

left_pair, right_pair = out_quad

top_right_mps_tensors.append(right_pair[0])
right_side_left_schmidt_values_list.append(right_pair[1])

top_left_mps_tensors.append(left_pair[0])
left_side_right_schmidt_values_list.append(left_pair[1])